In [10]:
import pandas as pd
from scipy.interpolate import interp1d

# File containing the supernova data
file_path = "Supernova_Ia/table4.dat"

# Step 1: Read the file into a pandas DataFrame
# Adjust `delim_whitespace=True` for space-separated values
columns = ["unimportant", "SN", "JD", "U_mag", "U_err", "B_mag", "B_err", "V_mag", "V_err", "R_mag", "R_err", "I_mag", "I_err", "Source"]
df = pd.read_csv(file_path, delim_whitespace=True, header=None, 
                 names=columns)

# Step 2: Group data by supernova identifier
grouped = df.groupby("SN")

# Define a function to process each supernova
def process_sn(group):
    # Step 2.1: Identify the peak magnitude
    peak_index = group["B_mag"].idxmin()
    peak_B_mag = group.loc[peak_index, "B_mag"]
    peak_JD = group.loc[peak_index, "JD"]
    
    # Step 2.2: Calculate B magnitude 15 days after the peak
    target_JD = peak_JD + 15
    jd_array = group["JD"].values
    bmag_array = group["B_mag"].values
    
    # Interpolation function
    interp_func = interp1d(jd_array, bmag_array, kind='linear', fill_value='extrapolate')
    b_15 = interp_func(target_JD)
    delta_m15 = b_15 - peak_B_mag
    
    # Step 2.3: Apply the Phillips relation
    M_B_0 = -19.3  # Fiducial peak absolute magnitude
    alpha = 0.7    # Phillips relation slope
    M_B = M_B_0 + alpha * (delta_m15 - 1.1)
    
    # Step 2.4: Calculate distance modulus and distance
    mu = peak_B_mag - M_B
    distance_pc = 10 ** ((mu + 5) / 5)  # Distance in parsecs
    distance_mpc = distance_pc / 1e6   # Distance in megaparsecs
    
    # Return results
    return {
        "SN": group["SN"].iloc[0],
        "Peak_B_mag": peak_B_mag,
        "Delta_m15": delta_m15,
        "M_B": M_B,
        "Distance_Mpc": distance_mpc
    }

# Step 3: Process each supernova group and collect results
results = []
for name, group in grouped:
    result = process_sn(group)
    results.append(result)

# Step 4: Create a summary DataFrame
results_df = pd.DataFrame(results)

# Save results to a CSV file
results_df.to_csv("supernova_results.csv", index=False)

print(results_df)

/tmp/ipykernel_6323/298767585.py:10: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(file_path, delim_whitespace=True, header=None,


UFuncTypeError: ufunc 'subtract' did not contain a loop with signature matching types (dtype('float64'), dtype('<U6')) -> None